<a href="https://colab.research.google.com/github/ds4geo/ds4geo/blob/master/WS%202020%20Course%20Notes/Session%207.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Science for Geoscientists - Winter Semester 2020**
# **Session 7 - Data Exploration - 18th November 2020**

# Part 7.1 - Warm-up Excercise - *Workshop*
Calculate the average value of the data in the following csv file:

https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/unordered/s7_warmup.csv


# Part 7.2 - Weather Assignment Review - *Discussion*

* General comments
* specific example of plotting by year - mention group-by
* Discussion about narative and story-telling
* "How do I do that?"

# Part 7.3 - Weather Assignment Part 2

Thank you and well done with your assignments from week 3 and the feedback you have given each other. There were many impressive analyses and good naratives, but all have room for improvement.

This assignment is to continue with your week 3 weather data analysis submissions. The general guidelines are as they were origionally, but additionally:
1. Expand your analysis by:
 * adding more cities
 and/or
 * adding additional analyses or visualisations
2. Focus on:
 * Creating an interesting and compelling story/narative with a clear introduction (hook), and conclusion at the end.
 * Creating clear and informative visualisations.
 * Creating a clear structure using text cells and headings.




**Submission**
* Submit the assignment here: https://github.com/ds4geo/ds4geo_ws2020/tree/master/Assignments/Session%207
* The **deadline** is 23:59 on 1st December 2020.
* This assignment comprises 10% of the assessment for the course. Marks are awarded for clear, effective and interesting data driven story telling.

Submitted notebooks will be available to the whole class.


# Part 7.4 - Interactive Data Exploration Excercise - *Discussion/Walkthrough*

We will explore the HydroLakes (https://www.hydrosheds.org/pages/hydrolakes) dataset containing information about 1.4 million lakes greater than 10 ha in size around the world.

Attribute data includes:
* Lake_name
* Country
* Continent
* Lake_type (1=natural, 2=dam, 3=natural with controls)
* Lake_area (in km2)
* Shore_len (length of shoreline in km)
* Shore_dev (measure of shape - 1=circle, higher = more non-circular, irregular or complex)
* Vol_total (lake volume in million cubic meters, 0.001 km3)
* Depth_avg (average lake depth in m)
* Dis_avg (average discharge through lake in cubic m/s)
* Res_time (average residence time in days)
* Elevation (meters above sea level)
* Slope_100 (average topographic slope in degrees in 100 m buffer of shoreline)
* Wshd_area (area of lake watershed in km2)
* Pour_lat, Pour_long (lat and long of lake pour point)
* Cent_lat, Cent_long (lat and long of lake centre point)

Students will suggest analyses and visualisations, and we will code line by line together to perform them.



## Part 7.4.1 - Loading the data
The data is contained in a series of csv files here:

https://github.com/ds4geo/ds4geo/tree/master/data/geospatial/HydroLakes

## Part 7.4.2 - Simple overviews and statistics


In [ ]:
# ideas:
'''
- total area, compared to land area and global surface area
- total volume, compared to something?
- top lakes by area, depth, other stats?
- etc.
'''

## Part 7.4.3 - Advanced analyses and visualisations

In [ ]:
# ideas:
'''
- where is the water surface concentrated?
- where are the shorelines concentrated?
- where is the water volume concentrated?

- relationships between parameters: depth, area, shoreline, residence time, location, etc.

'''